In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install transformers
!pip install simpletransformers

In [ ]:
# install simpleT5
!pip install simplet5

     |████████████████████████████████| 2.3MB 5.5MB/s 
     |████████████████████████████████| 808kB 45.9MB/s 
     |████████████████████████████████| 10.6MB 55.2MB/s 
     |████████████████████████████████| 829kB 50.5MB/s 
     |████████████████████████████████| 276kB 60.6MB/s 
     |████████████████████████████████| 645kB 44.7MB/s 
     |████████████████████████████████| 12.2MB 44.8MB/s 
     |████████████████████████████████| 4.5MB 49.5MB/s 
     |████████████████████████████████| 1.3MB 49.1MB/s 
     |████████████████████████████████| 143kB 64.0MB/s 
     |████████████████████████████████| 296kB 58.0MB/s 
  Created wheel for simplet5: filename=simplet5-0.0.9-cp37-none-any.whl size=6203 sha256=20de42b84902ec09e054785bfb1501b7c7dbc040b30832dd156f009cbf958c90
  Stored in directory: /root/.cache/pip/wheels/da/6d/98/dcb5b919cab92312876c4f5ceda5223ec51f66d59fa290da67
  Created wheel for fastt5: filename=fastt5-0.0.5-cp37-none-any.whl size=14574 sha256=6f33e6bc219fecf1dddf1c8f669541bace71

In [ ]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('1.9.0+cu102', '4.6.1', '0.10.3')

In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


##TroFi-X - Abstract Summarization (Noun+Verb)

In [ ]:
data_file = pd.read_csv("data/trofix.csv")

In [ ]:
data_file = data_file.rename(columns={'sentence': 'source_text'}) #abstract - sentence
data_file['target_text'] = data_file[['arg1', 'arg2', 'verb']].agg(' '.join, axis=1) #title - metaphorical words
#data_file['target_text'] = data_file['verb'] #title - metaphorical words

In [ ]:
papers = data_file[["source_text", "target_text"]]
papers.head()

,source_text,target_text
0,Triple mileage has struck another blow to the ...,mileage struck blow
1,U.S. officials said evidence suggests that a J...,terrorist attack target
2,"Some police forces , for example , have steppe...",forces stepped use
3,"Every day his troops gather under the green , ...",day pour stream
4,He says manufacturers are increasingly rolling...,manufacturers rolling products


In [ ]:
# Load test data - TroFi
import pandas as pd
# Use a subset for quick experiments
#data = data[:10000]

from sklearn.model_selection import train_test_split as tts
import pandas as pd

# Split to train, val and test
train_df, test_data = tts(data_file[["source_text", "target_text"]], random_state=42, test_size=0.1)
train_df, val = tts(train_df, random_state=42, test_size=test_data.shape[0])

In [ ]:
%%time

from simpletransformers.seq2seq import Seq2SeqModel

test_df = pd.read_csv("stockholm/bert_code/trofix_bert_subs/trofix_all_met.csv")
test_df = test_df.rename(columns={'sentence': 'source_text'}) #abstract - sentence
test_df['target_text'] = test_df[['arg1', 'arg2', 'verb']].agg(' '.join, axis=1) #title - metaphorical words

CPU times: user 1.94 s, sys: 259 ms, total: 2.19 s
Wall time: 3.86 s


In [ ]:
# import
from simplet5 import SimpleT5

# instatntiate
model = SimpleT5()

# load
model.from_pretrained("t5","t5-base")

# train
model.train(train_df=train_df, eval_df=test_df, source_max_token_len=512, target_max_token_len=128, max_epochs=5, batch_size=8, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42
INFO:filelock:Lock 139886642685776 acquired on /root/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

INFO:filelock:Lock 139886642685776 released on /root/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock
INFO:filelock:Lock 139886642686160 acquired on /root/.cache/huggingface/transformers/90de37880b5ff5ac7ab70ff0bd369f207e9b74133fa153c163d14c5bb0116207.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529.lock


Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

INFO:filelock:Lock 139886642686160 released on /root/.cache/huggingface/transformers/90de37880b5ff5ac7ab70ff0bd369f207e9b74133fa153c163d14c5bb0116207.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529.lock
INFO:filelock:Lock 139886615588624 acquired on /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637.lock


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
!ls outputs/

checkpoint-10000		    SimpleT5-epoch-2-train-loss-0.132
checkpoint-2000			    SimpleT5-epoch-2-train-loss-0.2453
checkpoint-4000			    SimpleT5-epoch-2-train-loss-0.2695
checkpoint-6000			    SimpleT5-epoch-2-train-loss-0.2735
checkpoint-8000			    SimpleT5-epoch-2-train-loss-0.3045
config.json			    SimpleT5-epoch-2-train-loss-0.3159
eval_results.txt		    SimpleT5-epoch-2-train-loss-0.3466
merges.txt			    SimpleT5-epoch-2-train-loss-0.3474
model_args.json			    SimpleT5-epoch-2-train-loss-0.3475
pytorch_model.bin		    SimpleT5-epoch-2-train-loss-0.787
SimpleT5-epoch-0-train-loss-0.8219  SimpleT5-epoch-3-train-loss-0.0384
SimpleT5-epoch-0-train-loss-0.858   SimpleT5-epoch-3-train-loss-0.0431
SimpleT5-epoch-0-train-loss-0.8869  SimpleT5-epoch-3-train-loss-0.0493
SimpleT5-epoch-0-train-loss-1.0937  SimpleT5-epoch-3-train-loss-0.0702
SimpleT5-epoch-0-train-loss-1.4472  SimpleT5-epoch-3-train-loss-0.1619
SimpleT5-epoch-0-train-loss-1.8205  SimpleT5-epoch-3-train-loss-0.1764
SimpleT5-epoch

In [ ]:
# load a trained model
model.load_model("outputs/SimpleT5-epoch-4-train-loss-1.1577", use_gpu=True)

In [ ]:
# let's see how it performerd:
sample_abstracts = test_df.sample(35)

for i, abstract in sample_abstracts.iterrows():
    print(f"===== Original Sentence =====")
    print(abstract['source_text'])
    summary= model.predict(abstract['source_text'])[0]
    print(f"\n===== Actual Metaphorical Tokens =====")
    print(f"{abstract['target_text']}")
    print(f"\n===== Predicted Metaphorical Tokens =====")
    print(f"{summary}")
    print("\n +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

===== Original Sentence =====
The issue is particularly hot in Florida , where a coalition of civic leaders , civil libertarians and Hispanic groups went to federal court recently in an unsuccessful attempt to knock the proposed amendment off the ballot .

===== Actual Metaphorical Tokens =====
attempt knock amendment

===== Predicted Metaphorical Tokens =====
attempt knock amendment

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

===== Original Sentence =====
He adds , `` We 're trying very hard to avoid the Halley 's comet syndrome -- where you grab everything in sight and then you vanish . ''

===== Actual Metaphorical Tokens =====
you grab everything

===== Predicted Metaphorical Tokens =====
you grab everything

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

===== Original Sentence =====
But Cameloot was just another loser among the entries that flooded the offices of Circus Circus Enterprises Inc. during a 